In [2]:
import theano
import numpy

from theano import tensor

theano.config.floatX = 'float32'

Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled, CuDNN 4007)


## Load model and parameters from vggnet 19

In [3]:
import lasagne
import pickle

from vggnet import build_model
vggnet_model = build_model()

params_val = pickle.load(open('./vgg19.pkl'))
lasagne.layers.set_all_param_values(vggnet_model['pool5'], params_val['param values'][:32])

## Define gram matrix function

In [4]:
def gram_matrix(X):
    X = X.flatten(ndim=3)
    return tensor.tensordot(X, X, axes=([2], [2]))

## Build the network for texture generation

In [5]:
X = tensor.ftensor4('image')

Zs = dict()
for i in range(5):
    Zs['Z%d' % (i+1,)] = tensor.ftensor4('Z%d' % (i+1,))

In [6]:
from tg_model import build_model

texture_generation_nn = build_model(Zs)
output = lasagne.layers.get_output(texture_generation_nn['output'], Zs)

In [7]:
f_texture_generation = theano.function([v for v in Zs.itervalues()], [output])

In [8]:
texture_loss_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']

def texture_features(X):
    layers = [vggnet_model[k] for k in texture_loss_layers]
    outputs = lasagne.layers.get_output(layers, X)
    
    return outputs

In [9]:
# test texture generation net

Z1 = numpy.random.rand(1, 1, 256, 256).astype('float32')
Z2 = numpy.random.rand(1, 1, 128, 128).astype('float32')
Z3 = numpy.random.rand(1, 1, 64, 64).astype('float32')
Z4 = numpy.random.rand(1, 1, 32, 32).astype('float32')
Z5 = numpy.random.rand(1, 1, 16, 16).astype('float32')

f_texture_generation(Z1, Z2, Z3, Z4, Z5)

ValueError: GpuJoin: Wrong inputs for input 2 related to inputs 0.!
Apply node that caused the error: GpuJoin(TensorConstant{1}, GpuReshape{4}.0, GpuElemwise{Composite{(i0 * ((i1 + i2) + Abs((i1 + i2))))}}[(0, 1)].0)
Toposort index: 484
Inputs types: [TensorType(int8, scalar), CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, 4D)]
Inputs shapes: [(), (1, 16, 512, 512), (1, 8, 16, 16)]
Inputs strides: [(), (0, 262144, 512, 1), (0, 256, 16, 1)]
Inputs values: [array(1, dtype=int8), 'not shown', 'not shown']
Outputs clients: [[GpuContiguous(GpuJoin.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

## Load an image

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

texture_image = mpimg.imread('./red-peppers256.jpg')
plt.imshow(texture_image)

## Compute texture features for target image

In [ ]:
texture_image_nn_input = numpy.rollaxis(texture_image, 2)[None]

f_features = theano.function(
        inputs=[X],
        outputs=texture_features(X)
)
texture_features = f_features(texture_image_nn_input)